In [1]:
import numpy as np
from array import *
import csv

# using datetime module
import datetime;

# Binary Search
from bisect import bisect_left
from bisect import bisect_right

import matplotlib.pyplot as plt
import json

import multiprocessing as mp
from multiprocessing import Process, Queue
from multiprocessing import current_process
import queue
import threading


import os
os.environ["PYARROW_IGNORE_TIMEZONE"] = "1"
from pyspark.sql import SparkSession
import pyspark.pandas as ps
from pyspark.sql.functions import col
import pandas as pd

import random
import networkx as nx
from tqdm import tqdm


#https://python-louvain.readthedocs.io/en/latest/api.html
#community.modularity(partition, graph, weight='weight')
from community import modularity



print('----------------------')
#unique_payment_addresses_len = len(unique_payment_addresses)
unique_raw_addresses_len        = 40330345
unique_payment_addresses_len    = 40324960
unique_delegation_addresses_len = 3868049
print('unique_raw_addresses_len        = ', unique_raw_addresses_len)
print('unique_payment_addresses_len    = ', unique_payment_addresses_len)
print('unique_delegation_addresses_len = ', unique_delegation_addresses_len)


print('----------------------')
print('done!')



----------------------
unique_raw_addresses_len        =  40330345
unique_payment_addresses_len    =  40324960
unique_delegation_addresses_len =  3868049
----------------------
done!


In [2]:
print('----------------------')

#print(os.path.basename("/path/to/some/file.txt"))
#print(os.path.dirname("/path/to/some/file.txt"))

BASE_ADDRESS = '/local/scratch/exported/blockchain_parsed/cardano_mostafa'
TEMP_ADDRESS = BASE_ADDRESS + '/temp_files/'


print('----------------------')
print('done!')


----------------------
----------------------
done!


In [3]:
# Define required methods:


print('----------------------')

# ct stores current time
ct = datetime.datetime.now()
print("current time: ", ct)



##########################################################################################
def parent (id1, parents_array):
    return parents_array[id1];

##########################################################################################
def find_parent (id1, parents_array):
    while (id1 != parent(id1, parents_array)):
        new_parent = parent(parent(id1, parents_array), parents_array);
        id1        = new_parent;
    
    return id1

##########################################################################################
# Link two addresses based on "Union-Find" Algorithm:
def link_address (addr_position_1, addr_position_2, parents_array):
    id1 = find_parent(addr_position_1, parents_array)
    id2 = find_parent(addr_position_2, parents_array)

    if (id1 == id2):
        return

    # make sure always we have id1 > id2
    if id1 < id2:
        x = id1
        id1 = id2
        id2 = x

    parents_array[id1] = id2
    return

##########################################################################################
def resolveAll (parents_array):
    for id1 in tqdm(range(len(parents_array))):
        parents_array[id1] = find_parent(id1, parents_array);
    return

##########################################################################################
def remapClusterIds (parents_array, clustering_array):
    cluster_count = 0
    place_holder = 9999999999999
    new_cluster_ids = [place_holder] * len(parents_array)
    
    for i in range(len(clustering_array)):
        clustering_array[i] = parents_array[i]

    for i in tqdm(range(len(clustering_array))):
        parent_index = clustering_array[i]

        if (new_cluster_ids [parent_index] == place_holder):
            new_cluster_ids [parent_index] = cluster_count
            cluster_count = cluster_count + 1

        clustering_array[i] = new_cluster_ids [parent_index]

    return cluster_count;

##########################################################################################
def merge_parents(parents_array, parents_merged):
    if (len(parents_array) != len(parents_merged)):
        print('parents_merged Error: -1 (Length)')
        return -1
    
    for i in tqdm(range(len(parents_merged))):
        link_address (i, parents_array[i], parents_merged)


##########################################################################################
def BinarySearch(a, x):
    i = bisect_left(a, x)
    if i < len(a) and a[i] == x:
        return i
    else:
        print('BinarySearch Error: -1')
        return -1

##########################################################################################
def BinarySearch_Find_start_end(a, x):
    i = bisect_left(a, x)
    j = bisect_right(a, x) - 1
    if i < len(a) and a[i] == x and j < len(a) and a[j] == x:
        return [i, j]
    else:
        print('BinarySearch Error: -1')
        print('i = ', i)
        print('j = ', j)
        return -1


##########################################################################################
def store_array_to_file (input_array_name, file_name, index_=False, header_=None):
    ct = datetime.datetime.now()
    curr_timestamp = str(ct)[0:10] + '_' + str(ct)[11:13] + str(ct)[14:16] + str(ct)[17:19]
    print('start time (Store Array to ' + file_name + '): ', ct)

    df = pd.DataFrame(input_array_name)
    df.to_csv(file_name, index=index_, header=header_)

    '''
    with open(file_name, 'w') as filehandle:
        for element in input_array_name:
            filehandle.write(f'{element}\n')
    '''

    et = datetime.datetime.now() - ct
    print('elapsed time (Store Array to ' + file_name + '): ', et)

    return

##########################################################################################
def load_file_to_array (file_name, header_=None):
    ct = datetime.datetime.now()
    curr_timestamp = str(ct)[0:10] + '_' + str(ct)[11:13] + str(ct)[14:16] + str(ct)[17:19]
    print('start time (Load ' + file_name  + ' to Array): ', ct)

    df = pd.read_csv(file_name, header=header_)
    output_array_name = df.to_numpy()
    
    '''
    output_array_name = []
    with open(file_name, 'r') as filehandle:
        for line in filehandle:
            # Remove linebreak which is the last character of the string
            curr_place = line[:-1]
            # Add item to the list
            output_array_name.append(curr_place)
    '''    

    '''
    for index, row in df.iterrows():
        new_line = df.loc[index , 0]
        output_array_name.append(new_line)
        if (index%1000000 == 0):
                print('One New Milion Records loaded: ', index)
    '''

    et = datetime.datetime.now() - ct
    print('elapsed time (Load ' + file_name  + ' to Array): ', et)
    
    return output_array_name

##########################################################################################
def store_array_to_file_2D (input_array_name, file_name):
    ct = datetime.datetime.now()
    curr_timestamp = str(ct)[0:10] + '_' + str(ct)[11:13] + str(ct)[14:16] + str(ct)[17:19]
    print('start time (Store Array 2D to ' + file_name + '): ', ct)

    with open(file_name, "w") as filehandle:
        json.dump(input_array_name, filehandle)
    
    et = datetime.datetime.now() - ct
    print('elapsed time (Store Array 2D to ' + file_name + '): ', et)

    return

##########################################################################################
def load_file_to_array_2D (file_name):
    ct = datetime.datetime.now()
    curr_timestamp = str(ct)[0:10] + '_' + str(ct)[11:13] + str(ct)[14:16] + str(ct)[17:19]
    print('start time (Load ' + file_name  + ' to Array 2D): ', ct)

    with open(file_name) as filehandle:
        output_array_name = json.load(filehandle)

    et = datetime.datetime.now() - ct
    print('elapsed time (Load ' + file_name  + ' to Array 2D): ', et)
    
    return output_array_name


##########################################################################################
def store_dict_to_file_INT (input_dict_name, file_name):
    ct = datetime.datetime.now()
    curr_timestamp = str(ct)[0:10] + '_' + str(ct)[11:13] + str(ct)[14:16] + str(ct)[17:19]
    print('start time (Store Dictionary to ' + file_name + '): ', ct)

    filehandle = csv.writer(open(file_name, 'w'))
    for key, val in input_dict_name.items():
        filehandle.writerow([key, val])

    et = datetime.datetime.now() - ct
    print('elapsed time (Store Dictionary to ' + file_name + '): ', et)

    return

##########################################################################################
def load_file_to_dict_INT (file_name):
    ct = datetime.datetime.now()
    curr_timestamp = str(ct)[0:10] + '_' + str(ct)[11:13] + str(ct)[14:16] + str(ct)[17:19]
    print('start time (Load ' + file_name  + ' to Dictionary): ', ct)

    filehandle = csv.reader(open(file_name, 'r'))
    output_dict_name = {int(rows[0]):int(rows[1]) for rows in filehandle}

    et = datetime.datetime.now() - ct
    print('elapsed time (Load ' + file_name  + ' to Dictionary): ', et)
    
    return output_dict_name

##########################################################################################
def add_edge_info(node_1, node_2, edges_array, weight=1):
    if (node_1 == node_2):
        return

    # make sure always we have n1 > n2:
    if (node_1 < node_2):
        n1 = node_2
        n2 = node_1
    else:
        n1 = node_1
        n2 = node_2

    for i in range(weight):
        edges_array[n1].append(n2)

    return

##########################################################################################
def extract_payment_delegation_parts(address_raw, payment_cred, stake_address):
    if (address_raw == ''):
        #print(' - Error: address_raw is empty!')
        return ['', '']

    if (address_raw[2] == '8'): #Byron Address
        if (payment_cred != ''):
            print(' - Error: payment_cred in Byron Address is NOT empty!')
            return ['', '']
        if (stake_address != ''):
            print(' - Error: stake_address in Byron Address is NOT empty!')
            return ['', '']
        payment_part    = address_raw
        delegation_part = ''

    else: #Shelley Address
        if (payment_cred == ''):
            print(' - Error: payment_cred in Shelley Address is empty!')
            return ['', '']
        payment_part    = payment_cred
        delegation_part = stake_address

    return [payment_part, delegation_part]

##########################################################################################
# Function which calculates the Gini index
# Inputs: array with length equal to number of agents; each element in array represents the wealth of the agent
# Outputs: gini index of the system
def gini_index(inp_array):
    array = np.array(inp_array)
    array = array.astype(float)
    array = array.flatten() #all values are treated equally, arrays must be 1d
    if np.amin(array) < 0:
        array -= np.amin(array) #values cannot be negative
    array += 0.0000001 #values cannot be 0
    array = np.sort(array) #values must be sorted
    index = np.arange(1,array.shape[0]+1) #index per array element
    n = array.shape[0] #number of array elements
    return ((np.sum((2 * index - n  - 1) * array)) / (n * np.sum(array))) #Gini coefficient

##########################################################################################



##########################################################################################



##########################################################################################
print('----------------------')
print('done!')



----------------------
current time:  2023-04-07 09:44:45.172985
----------------------
done!


In [ ]:
# Create List of all Addresses [raw_address_list, payment_address_list, and delegation_address_list]:

'''
print('----------------------')

# ct stores current time
ct = datetime.datetime.now()
print("current time: ", ct)
print('----------------------')


# List of all addresses (from INPUTs and OUTPUTs)
raw_address_list = []
payment_address_list = []
delegation_address_list = []



CSV_FILES_NAME_FORMAT = BASE_ADDRESS + '/cardano_TXs_'
NUMBER_OF_CSV_FILES = 6
CSV_FILES_SUFFIX = '.csv'


for i in range(1, NUMBER_OF_CSV_FILES + 1):

    ct_temp = datetime.datetime.now()

    file_name = CSV_FILES_NAME_FORMAT + str(i) + CSV_FILES_SUFFIX
    df = pd.read_csv(file_name, delimiter='|')

    et_temp = datetime.datetime.now() - ct_temp
    print("elapsed time (Load CSV File " + file_name + "): ", et_temp)

    ct_temp = datetime.datetime.now()

    for index, row in tqdm(df.iterrows()):
        ##########################################################################################
        #inputs_list = list( df.loc[index , 'INPUTs'].split(';') )
        #for tx_input in inputs_list:
        #    tx_input_UTXO_address = tx_input.split(',')[4]
        #    payment_address_list.append(tx_input_UTXO_address)
        ##########################################################################################
        outputs_list = list( df.loc[index , 'OUTPUTs'].split(';') )
        #TX_ID = df.loc[index , 'TX_ID']
        for tx_output in outputs_list:
            address_raw        = tx_output.split(',')[1]
            address_has_script = tx_output.split(',')[4]
            payment_cred       = tx_output.split(',')[5]
            stake_address      = tx_output.split(',')[6]
            [address_payment_part, address_delegation_part] = extract_payment_delegation_parts(address_raw, payment_cred, stake_address)
            if (address_raw != ''):
                raw_address_list.append(address_raw)
            if (address_payment_part != ''):
                payment_address_list.append(address_payment_part)
            if (address_delegation_part != ''):
                delegation_address_list.append(address_delegation_part)
        ##########################################################################################

    et_temp = datetime.datetime.now() - ct_temp
    print("elapsed time (Extract Addresses from INs/OUTs of CSV File " + file_name + "): ", et_temp)



print('----------------------')
et = datetime.datetime.now() - ct
print("Total elapsed time (Create list of Addresses Array List in Python): ", et)


print('----------------------')
print('done!')
'''



In [ ]:
# Write "raw_address_list/payment_address_list/delegation_address_list" into a File:

'''
print('----------------------')

ct = datetime.datetime.now()
print("current time: ", ct)


# write a list into a file:
ct = datetime.datetime.now()
curr_timestamp = str(ct)[0:10] + '_' + str(ct)[11:13] + str(ct)[14:16] + str(ct)[17:19]
output_filename = BASE_ADDRESS + '/AddressListRaw__Cardano_TXs_All__' + curr_timestamp + '.txt'
store_array_to_file (raw_address_list, output_filename)



# write a list into a file:
ct = datetime.datetime.now()
curr_timestamp = str(ct)[0:10] + '_' + str(ct)[11:13] + str(ct)[14:16] + str(ct)[17:19]
output_filename = BASE_ADDRESS + '/AddressListPayment__Cardano_TXs_All__' + curr_timestamp + '.txt'
store_array_to_file (payment_address_list, output_filename)



# write a list into a file:
ct = datetime.datetime.now()
curr_timestamp = str(ct)[0:10] + '_' + str(ct)[11:13] + str(ct)[14:16] + str(ct)[17:19]
output_filename = BASE_ADDRESS + '/AddressListDelegation__Cardano_TXs_All__' + curr_timestamp + '.txt'
store_array_to_file (delegation_address_list, output_filename)



print('----------------------')
print('done!')
'''


In [ ]:
# In Linux ("Sort" and "Remove Duplicate Addresses" in raw_address_list/payment_address_list/delegation_address_list:


'''
print('----------------------')
ct = datetime.datetime.now()
print("current time: ", ct)

os.system('sort -k 1 -u /local/scratch/exported/blockchain_parsed/cardano_mostafa/AddressListRaw__Cardano_TXs_All__2023-02-28_143357.txt        > /local/scratch/exported/blockchain_parsed/cardano_mostafa/Unique_AddressesListRaw__Cardano_TXs_All__2023-02-28_143357.txt')
os.system('sort -k 1 -u /local/scratch/exported/blockchain_parsed/cardano_mostafa/AddressListPayment__Cardano_TXs_All__2023-02-28_143953.txt    > /local/scratch/exported/blockchain_parsed/cardano_mostafa/Unique_AddressesListPayment__Cardano_TXs_All__2023-02-28_143953.txt')
os.system('sort -k 1 -u /local/scratch/exported/blockchain_parsed/cardano_mostafa/AddressListDelegation__Cardano_TXs_All__2023-02-28_144415.txt > /local/scratch/exported/blockchain_parsed/cardano_mostafa/Unique_AddressesListDelegation__Cardano_TXs_All__2023-02-28_144415.txt')

print('----------------------')
print('done!')
'''



In [4]:
# Read ("sorted" "unique" array_list) [raw_address_list/payment_address_list/delegation_address_list] from file:

print('----------------------')


file_name = BASE_ADDRESS + '/Unique_AddressesListRaw__Cardano_TXs_All__2023-02-28_143357.txt'
unique_raw_addresses = load_file_to_array (file_name)
print('Length of \"unique_raw_addresses\" = ' + str(len(unique_raw_addresses)))


file_name = BASE_ADDRESS + '/Unique_AddressesListPayment__Cardano_TXs_All__2023-02-28_143953.txt'
unique_payment_addresses = load_file_to_array (file_name)
print('Length of \"unique_payment_addresses\" = ' + str(len(unique_payment_addresses)))


file_name = BASE_ADDRESS + '/Unique_AddressesListDelegation__Cardano_TXs_All__2023-02-28_144415.txt'
unique_delegation_addresses = load_file_to_array (file_name)
print('Length of \"unique_delegation_addresses\" = ' + str(len(unique_delegation_addresses)))


##########################################################################################
print('----------------------')
print('done!')



----------------------
start time (Load /local/scratch/exported/blockchain_parsed/cardano_mostafa/Unique_AddressesListRaw__Cardano_TXs_All__2023-02-28_143357.txt to Array):  2023-04-07 09:44:58.531741
elapsed time (Load /local/scratch/exported/blockchain_parsed/cardano_mostafa/Unique_AddressesListRaw__Cardano_TXs_All__2023-02-28_143357.txt to Array):  0:00:46.270749
Length of "unique_raw_addresses" = 40330345
start time (Load /local/scratch/exported/blockchain_parsed/cardano_mostafa/Unique_AddressesListPayment__Cardano_TXs_All__2023-02-28_143953.txt to Array):  2023-04-07 09:45:44.802923
elapsed time (Load /local/scratch/exported/blockchain_parsed/cardano_mostafa/Unique_AddressesListPayment__Cardano_TXs_All__2023-02-28_143953.txt to Array):  0:00:35.430624
Length of "unique_payment_addresses" = 40324960
start time (Load /local/scratch/exported/blockchain_parsed/cardano_mostafa/Unique_AddressesListDelegation__Cardano_TXs_All__2023-02-28_144415.txt to Array):  2023-04-07 09:46:20.234022


In [5]:
# count Byron and Shelley addresses:

byron_count = 0
shelley_count = 0

for i in tqdm(range(len(unique_raw_addresses))):
    if (unique_raw_addresses[i][0][2] == '8'): #Byron Address
        byron_count = byron_count+1
    else: #Shelley Address
        shelley_count = shelley_count+1

print('byron_count   = ', byron_count)
print('shelley_count = ', shelley_count)


##########################################################################################
print('----------------------')
print('done!')



100%|██████████| 40330345/40330345 [00:21<00:00, 1853957.05it/s]

byron_count   =  11282384
shelley_count =  29047961
----------------------
done!


In [ ]:
# Find Number of new Byron, Shelley, and Stake Addresses VS "Time":

'''
print('----------------------')
# ct stores current time
ct = datetime.datetime.now()
print("current time: ", ct)


print('----------------------')
file_name = BASE_ADDRESS + '/cardano_epochs_MinTXID_CardanoAllTXs.csv'
df = pd.read_csv(file_name, delimiter='|')
#df['EPOCH_NO']
#df['MIN_TX_ID']
#df['MIN_TIME_UTC']
first_TX_IDs_in_epoch_array = df['MIN_TX_ID'].to_numpy()
current_epoch = -1



place_holder = 999999999999
raw_addresses_epoch_array             = np.array([place_holder] * len(unique_raw_addresses))
Byron_payment_addresses_epoch_array   = np.array([place_holder] * len(unique_payment_addresses))
Shelley_payment_addresses_epoch_array = np.array([place_holder] * len(unique_payment_addresses))
delegation_addresses_epoch_array      = np.array([place_holder] * len(unique_delegation_addresses))



CSV_FILES_NAME_FORMAT = BASE_ADDRESS + '/cardano_TXs_'
NUMBER_OF_CSV_FILES = 6
CSV_FILES_SUFFIX = '.csv'

for i in range(1, NUMBER_OF_CSV_FILES + 1):

    ct_temp = datetime.datetime.now()

    file_name = CSV_FILES_NAME_FORMAT + str(i) + CSV_FILES_SUFFIX
    df = pd.read_csv(file_name, delimiter='|')

    et_temp = datetime.datetime.now() - ct_temp
    print("elapsed time (Load CSV File " + file_name + "): ", et_temp)

    ct_temp = datetime.datetime.now()

    for index, row in tqdm(df.iterrows()):
        ##########################################################################################
        TX_ID = df.loc[index, 'TX_ID']
        if(TX_ID == first_TX_IDs_in_epoch_array[current_epoch + 2]):
            current_epoch = current_epoch + 1
        ##########################################################################################
        #inputs_list = list( df.loc[index , 'INPUTs'].split(';') )
        #for tx_input in inputs_list:
        #    tx_input_UTXO_address = tx_input.split(',')[4]
        #    payment_address_list.append(tx_input_UTXO_address)
        ##########################################################################################
        outputs_list = list( df.loc[index , 'OUTPUTs'].split(';') )
        #TX_ID = df.loc[index , 'TX_ID']
        for tx_output in outputs_list:
            address_raw        = tx_output.split(',')[1]
            address_has_script = tx_output.split(',')[4]
            payment_cred       = tx_output.split(',')[5]
            stake_address      = tx_output.split(',')[6]
            [address_payment_part, address_delegation_part] = extract_payment_delegation_parts(address_raw, payment_cred, stake_address)
            if (address_raw != ''): 
                addr_position = BinarySearch(unique_raw_addresses, address_raw)
                if(raw_addresses_epoch_array[addr_position] == place_holder):
                    raw_addresses_epoch_array[addr_position] = current_epoch

            if (address_payment_part != ''): 
                if (address_raw[2] == '8'): #Byron Address
                    addr_position = BinarySearch(unique_payment_addresses, address_payment_part)
                    if(Byron_payment_addresses_epoch_array[addr_position] == place_holder):
                        Byron_payment_addresses_epoch_array[addr_position] = current_epoch
                else: #Shelley Address
                    addr_position = BinarySearch(unique_payment_addresses, address_payment_part)
                    if(Shelley_payment_addresses_epoch_array[addr_position] == place_holder):
                        Shelley_payment_addresses_epoch_array[addr_position] = current_epoch

            if(address_delegation_part != ''):
                addr_position = BinarySearch(unique_delegation_addresses, address_delegation_part)
                if(delegation_addresses_epoch_array[addr_position] == place_holder):
                    delegation_addresses_epoch_array[addr_position] = current_epoch

        ##########################################################################################

    et_temp = datetime.datetime.now() - ct_temp
    print("elapsed time (Detect Byron and Shelley Addresses from INs/OUTs of CSV File " + file_name + "): ", et_temp)



print('----------------------')
et = datetime.datetime.now() - ct
print("Total elapsed time (Detect Byron and Shelley Addresses): ", et)


print('----------------------')
print('done!')
'''



In [ ]:
# Load/Store "raw_addresses_epoch_array"              from/into file:
#            "Byron_payment_addresses_epoch_array", 
#            "Shelley_payment_addresses_epoch_array", 
#            "delegation_addresses_epoch_array"


print('----------------------')
      

# Store "raw_addresses_epoch_array",              into file:
#       "Byron_payment_addresses_epoch_array", 
#       "Shelley_payment_addresses_epoch_array", 
#       "delegation_addresses_epoch_array"
'''
ct = datetime.datetime.now()
curr_timestamp = str(ct)[0:10] + '_' + str(ct)[11:13] + str(ct)[14:16] + str(ct)[17:19]

output_filename = BASE_ADDRESS + '/epochArray_rawAddresses__Cardano_TXs_All__' + curr_timestamp + '.txt'
print('output_filename = ', output_filename)
store_array_to_file(raw_addresses_epoch_array, output_filename)

output_filename = BASE_ADDRESS + '/epochArray_ByronAddresses__Cardano_TXs_All__' + curr_timestamp + '.txt'
print('output_filename = ', output_filename)
store_array_to_file(Byron_payment_addresses_epoch_array, output_filename)

output_filename = BASE_ADDRESS + '/epochArray_ShelleyAddresses__Cardano_TXs_All__' + curr_timestamp + '.txt'
print('output_filename = ', output_filename)
store_array_to_file(Shelley_payment_addresses_epoch_array, output_filename)

output_filename = BASE_ADDRESS + '/epochArray_delegationAddresses__Cardano_TXs_All__' + curr_timestamp + '.txt'
print('output_filename = ', output_filename)
store_array_to_file(delegation_addresses_epoch_array, output_filename)
'''



# Load  "raw_addresses_epoch_array",              from file:
#       "Byron_payment_addresses_epoch_array", 
#       "Shelley_payment_addresses_epoch_array", 
#       "delegation_addresses_epoch_array"
'''
file_name = BASE_ADDRESS + '/epochArray_rawAddresses__Cardano_TXs_All__?????????.txt'
raw_addresses_epoch_array             = load_file_to_array(file_name)

file_name = BASE_ADDRESS + '/epochArray_ByronAddresses__Cardano_TXs_All__?????????.txt'
Byron_payment_addresses_epoch_array   = load_file_to_array(file_name)

file_name = BASE_ADDRESS + '/epochArray_ShelleyAddresses__Cardano_TXs_All__?????????.txt'
Shelley_payment_addresses_epoch_array = load_file_to_array(file_name)

file_name = BASE_ADDRESS + '/epochArray_delegationAddresses__Cardano_TXs_All__?????????.txt'
delegation_addresses_epoch_array      = load_file_to_array(file_name)
'''

##########################################################################################
print('----------------------')
print('done!')


In [ ]:
# Generate "Parents_Array" based on Heuristics:


def generate_parents_array(queue_):
    # read input queue arguments
    in_args = queue_.get()

    csv_file_name = in_args[0]
    Heuristic_1   = in_args[1]
    Heuristic_2   = in_args[2]
    Heuristic_3   = in_args[3]

    csv_file_basename = os.path.basename(csv_file_name)

    # print current process identity
    str_current_proc = 'current_process()._identity[0] ' + '(' + csv_file_basename + ')' + ' = ' + str(current_process()._identity[0])
    print(str_current_proc)


    ct_temp = datetime.datetime.now()


    # Create SparkSession 
    spark = SparkSession.builder \
                     .master("local[1]") \
                     .appName("Mostafa_SparkTest_1") \
                     .config('spark.driver.maxResultSize', '70g') \
                     .config('spark.executor.cores', 4) \
                     .config('spark.executor.memory', '30g') \
                     .config('spark.driver.memory', '30g') \
                     .config('spark.memory.offHeap.enabled', True) \
                     .config('spark.memory.offHeap.size', '40g') \
                     .getOrCreate() 

    #df = pd.read_csv(csv_file_name, delimiter='|')
    df = spark.read.option("delimiter", "|").csv(csv_file_name, inferSchema=True, header=True)


    et_temp = datetime.datetime.now() - ct_temp
    print("elapsed time (Load CSV File " + csv_file_name + "): ", et_temp)


    ct_temp = datetime.datetime.now()


    # Initialize parents_array:
    parents_array = np.array([0] * unique_payment_addresses_len)
    for i in range(unique_payment_addresses_len):
        parents_array[i] = i    


    # Initialize graph_edges_array:
    graph_edges_array = [[] for _ in range(unique_payment_addresses_len)]


    #for index, row in df.iterrows():
    #position=int(str(csv_file_basename)[12:13]
    #for row in tqdm(df.collect(), desc=f'Process {csv_file_basename}', position=current._identity[0] - 1, leave=True):
    #for row in tqdm(df.collect(), desc=f'Process {csv_file_basename}'):
    for row in df.collect():
        #inputs_list = list( df.loc[index , 'INPUTs'].split(';') )
        #outputs_list = list( df.loc[index , 'OUTPUTs'].split(';') )
        inputs_list  = list( row['INPUTs'].split(';') )
        outputs_list = list( row['OUTPUTs'].split(';') )
        #TX_ID        = row['TX_ID']

        ##########################################################################################
        #Heuristic 1:
        if (Heuristic_1 == True):
            nonSC_addr_positions = []

            for i in range(0, len(inputs_list)):
                address_has_script = inputs_list[i].split(',')[7]
                if (address_has_script == 'f'): # non-Smart Contract Address
                    address_raw   = inputs_list[i].split(',')[4]
                    payment_cred  = inputs_list[i].split(',')[8]
                    stake_address = inputs_list[i].split(',')[9]
                    [address_payment_part, address_delegation_part] = extract_payment_delegation_parts(address_raw, payment_cred, stake_address)
                    if (address_payment_part != ''):
                        address_position = BinarySearch(unique_payment_addresses, address_payment_part)
                        nonSC_addr_positions.append(address_position) 

            for i in range(1, len(nonSC_addr_positions)):
                link_address(nonSC_addr_positions[0], nonSC_addr_positions[i], parents_array)                
                for j in range(0, i):
                    # find and store corresponding "edges" for Graph:
                    add_edge_info(node_1=nonSC_addr_positions[i], node_2=nonSC_addr_positions[j], edges_array=graph_edges_array, weight=1)


        ##########################################################################################
        #Heuristic 2 (link input[0] to outputs with values smaller than all inputs):
        if (Heuristic_2 == True):
            smallest_intput_UTXO_value = int(inputs_list[0].split(',')[6])
            for i in range(1, len(inputs_list)):
                smallest_intput_UTXO_value = min(smallest_intput_UTXO_value, int(inputs_list[i].split(',')[6]))

            for i in range(0, len(outputs_list)):
                tx_output_UTXO_value = int(outputs_list[i].split(',')[3])

                if (tx_output_UTXO_value < smallest_intput_UTXO_value):
                    first_input_UTXO_address = inputs_list[0].split(',')[4]
                    #addr_position_1 = np.where(unique_payment_addresses == first_input_UTXO_address)[0][0]
                    addr_position_1 = BinarySearch(unique_payment_addresses, first_input_UTXO_address)

                    tx_output_UTXO_address = outputs_list[i].split(',')[1]
                    #addr_position_2 = np.where(unique_payment_addresses == tx_output_UTXO_address)[0][0]
                    addr_position_2 = BinarySearch(unique_payment_addresses, tx_output_UTXO_address)

                    link_address(addr_position_1, addr_position_2, parents_array)
        
                    # find and store corresponding "edges" for Graph:
                    add_edge_info(node_1=addr_position_1, node_2=addr_position_2, edges_array=graph_edges_array, weight=1)
        




    spark.stop()


    # Resolve parents array
    resolveAll (parents_array)

    # Put file address of parents_array in queue
    ct_file = datetime.datetime.now()
    curr_timestamp = str(ct_file)[0:10] + '_' + str(ct_file)[11:13] + str(ct_file)[14:16] + str(ct_file)[17:26]

    output_parents_filename = TEMP_ADDRESS + '/parentsList_temp__' + csv_file_basename + '__' + curr_timestamp + '.txt'
    store_array_to_file   (parents_array, output_parents_filename)

    output_graghEdges_filename = TEMP_ADDRESS + '/graphEdgesList_temp__' + csv_file_basename + '__' + curr_timestamp + '.txt'
    store_array_to_file_2D(graph_edges_array, output_graghEdges_filename)

    queue_.put([output_parents_filename, output_graghEdges_filename])


    et_temp = datetime.datetime.now() - ct_temp
    print("elapsed time (Link Addresses with Heuristics in CSV File " + csv_file_basename + "): ", et_temp)
    
    return


##########################################################################################
print('----------------------')
print('done!')



In [ ]:
# Generate "balances_array" [Sequential Computation]:

'''
print('----------------------')

# ct stores current time
ct = datetime.datetime.now()
print("current time: ", ct)



CSV_FILES_NAME_FORMAT = BASE_ADDRESS + '/cardano_TXs_'
NUMBER_OF_CSV_FILES = 6
CSV_FILES_SUFFIX = '.csv'


# Initialize balances_array:
balances_array = np.array([0] * unique_payment_addresses_len)
for i in range(unique_payment_addresses_len):
    balances_array[i] = 0 



gini_array = [0]
gini_sample_id = 200000


for i in range(1, NUMBER_OF_CSV_FILES + 1):

    ct_temp = datetime.datetime.now()

    file_name = CSV_FILES_NAME_FORMAT + str(i) + CSV_FILES_SUFFIX
    df = pd.read_csv(file_name, delimiter='|')

    et_temp = datetime.datetime.now() - ct_temp
    print("elapsed time (Load CSV File " + file_name + "): ", et_temp)



    ct_temp = datetime.datetime.now()


    for index, row in tqdm(df.iterrows()):
        ##########################################################################################
        TX_ID = df.loc[index , 'TX_ID']
        inputs_list = list( df.loc[index , 'INPUTs'].split(';') )
        outputs_list = list( df.loc[index , 'OUTPUTs'].split(';') )

        #################################################################
        # Calculate "Gini Index":
        if (int(TX_ID) > gini_sample_id):
            gini_array_indx = int(gini_sample_id/200000)
            balances_array_no_zeros = balances_array[balances_array != 0]
            gini_array.append(gini_index(balances_array_no_zeros))
            if (TX_ID < 2000002):
                print('TX_ID = ', TX_ID, '  |  gini_array [', gini_array_indx, '] = ', gini_array[gini_array_indx])
            gini_sample_id = gini_sample_id + 200000

        #################################################################
        for i in range(0, len(inputs_list)):
            address_has_script = inputs_list[i].split(',')[7]
            if (address_has_script == 'f'): # non-Smart Contract Address
                address_raw   = inputs_list[i].split(',')[4]
                payment_cred  = inputs_list[i].split(',')[8]
                stake_address = inputs_list[i].split(',')[9]
                [address_payment_part, address_delegation_part] = extract_payment_delegation_parts(address_raw, payment_cred, stake_address)
                if (address_payment_part != ''):
                    address_position = BinarySearch(unique_payment_addresses, address_payment_part)
                    UTXO_value = inputs_list[i].split(',')[6]
                    balances_array[address_position] = balances_array[address_position] - int(UTXO_value)
        #################################################################
        for i in range(0, len(outputs_list)):
            address_has_script = outputs_list[i].split(',')[4]
            if (address_has_script == 'f'): # non-Smart Contract Address
                address_raw   = outputs_list[i].split(',')[1]
                payment_cred  = outputs_list[i].split(',')[5]
                stake_address = outputs_list[i].split(',')[6]
                [address_payment_part, address_delegation_part] = extract_payment_delegation_parts(address_raw, payment_cred, stake_address)
                if (address_payment_part != ''):
                    address_position = BinarySearch(unique_payment_addresses, address_payment_part)
                    UTXO_value = outputs_list[i].split(',')[3]
                    balances_array[address_position] = balances_array[address_position] + int(UTXO_value)
        #################################################################


    ##########################################################################################
    et_temp = datetime.datetime.now() - ct_temp
    print("elapsed time (Load CSV File " + file_name + "): ", et_temp)
    ##########################################################################################


##########################################################################################
print('----------------------')
print('done!')
'''



In [ ]:
# Store "balances_array" into file:
'''
print('----------------------')

ct = datetime.datetime.now()
curr_timestamp = str(ct)[0:10] + '_' + str(ct)[11:13] + str(ct)[14:16] + str(ct)[17:19]
output_filename = BASE_ADDRESS + '/balancesList_paymentAddresses_noSC__Cardano_TXs_All__' + curr_timestamp + '.txt'
print('output_filename = ', output_filename)

store_array_to_file (balances_array, output_filename)

##########################################################################################
print('----------------------')
print('done!')
'''


In [ ]:
# Store "gini_array" into file:

'''
print('----------------------')

ct = datetime.datetime.now()
curr_timestamp = str(ct)[0:10] + '_' + str(ct)[11:13] + str(ct)[14:16] + str(ct)[17:19]
output_filename = BASE_ADDRESS + '/giniArray_noZeros__paymentAddresses_noSC__Cardano_TXs_All__' + curr_timestamp + '.txt'
print('output_filename = ', output_filename)

store_array_to_file (gini_array, output_filename)

##########################################################################################
print('----------------------')
print('done!')
'''


In [ ]:
# Heuristic 2 (link "Shelley Addresses" with the same "address_delegation_part"):

'''
print('----------------------')

# ct stores current time
ct = datetime.datetime.now()
print("current time: ", ct)
print('----------------------')


# Initialize stake_delegation_array (For every "stake_address", this array shows which "payment_addresses" are delegated to it):
stake_delegation_array = [[] for _ in range(unique_delegation_addresses_len)]


CSV_FILES_NAME_FORMAT = BASE_ADDRESS + '/cardano_TXs_'
NUMBER_OF_CSV_FILES = 6
CSV_FILES_SUFFIX = '.csv'


for i in range(1, NUMBER_OF_CSV_FILES + 1):

    ct_temp = datetime.datetime.now()

    file_name = CSV_FILES_NAME_FORMAT + str(i) + CSV_FILES_SUFFIX
    df = pd.read_csv(file_name, delimiter='|')

    et_temp = datetime.datetime.now() - ct_temp
    print("elapsed time (Load CSV File " + file_name + "): ", et_temp)

    ct_temp = datetime.datetime.now()

    for index, row in tqdm(df.iterrows()):
        ##########################################################################################
        #inputs_list = list( df.loc[index , 'INPUTs'].split(';') )
        #for tx_input in inputs_list:
        #    tx_input_UTXO_address = tx_input.split(',')[4]
        #    payment_address_list.append(tx_input_UTXO_address)
        ##########################################################################################
        outputs_list = list( df.loc[index , 'OUTPUTs'].split(';') )
        #TX_ID = df.loc[index , 'TX_ID']
        for tx_output in outputs_list:
            address_raw        = tx_output.split(',')[1]
            address_has_script = tx_output.split(',')[4]
            payment_cred       = tx_output.split(',')[5]
            stake_address      = tx_output.split(',')[6]
            [address_payment_part, address_delegation_part] = extract_payment_delegation_parts(address_raw, payment_cred, stake_address)
            if (address_payment_part != '' and address_delegation_part != ''):
                indx1 = BinarySearch(unique_delegation_addresses, address_delegation_part)
                indx2 = BinarySearch(unique_payment_addresses, address_payment_part)
                stake_delegation_array[indx1].append(indx2)
        ##########################################################################################

    et_temp = datetime.datetime.now() - ct_temp
    print("elapsed time (Extract stake delegations from CSV File " + file_name + "): ", et_temp)



# unique sort the "stake_delegation_array":
for i in tqdm(range(len(stake_delegation_array))):
    stake_delegation_array[i] = sorted(set(stake_delegation_array[i]))


print('----------------------')
et = datetime.datetime.now() - ct
print("Total elapsed time (Heuristic2: find \"Shelley Addresses\" with the same \"address_delegation_part\"): ", et)


print('----------------------')
print('done!')
'''



In [ ]:
# Load/Store "stake_delegation_array" from/into file:

print('----------------------')


# Store "stake_delegation_array" into file:
'''
ct = datetime.datetime.now()
curr_timestamp = str(ct)[0:10] + '_' + str(ct)[11:13] + str(ct)[14:16] + str(ct)[17:19]
output_filename = BASE_ADDRESS + '/stakeDelegationArray__Heuristic2__Cardano_TXs_All__' + curr_timestamp + '.txt'
print('output_filename = ', output_filename)
store_array_to_file_2D(stake_delegation_array, output_filename)
'''



# Load stake_delegation_array from file:
'''
file_name = BASE_ADDRESS + '/stakeDelegationArray__Heuristic2__Cardano_TXs_All__2023-03-26_043620.txt'
stake_delegation_array = load_file_to_array_2D(file_name)
'''

##########################################################################################
print('----------------------')
print('done!')


In [ ]:
# Create and Fill "parents_heur2_array" (based on "stake_delegation_array"):

'''
print('----------------------')
ct = datetime.datetime.now()


# Initialize parents_array:
parents_heur2_array = np.array([0] * unique_payment_addresses_len)
for i in range(unique_payment_addresses_len):
    parents_heur2_array[i] = i


# link "Shelley Addresses" with the same "address_delegation_part":
for i in tqdm(range(len(stake_delegation_array))):
    for j in range(1, len(stake_delegation_array[i])):
        link_address(stake_delegation_array[i][0], stake_delegation_array[i][j], parents_heur2_array)


# Resolve parents array
resolveAll (parents_heur2_array)

print('----------------------')
et = datetime.datetime.now() - ct
print("Total elapsed time (Create and Fill \"heur2_parents_array\"): ", et)

##########################################################################################
print('----------------------')
print('done!')
'''



In [ ]:
# Create and Fill "Parents_" arrays (related to "Heuristic1"):

print('----------------------')

# ct stores current time
ct = datetime.datetime.now()
print("current time: ", ct)


##########################################################################################
if __name__ == "__main__":  # confirms that the code is under main function
    q1 = Queue()
    q2 = Queue()
    q3 = Queue()
    q4 = Queue()
    q5 = Queue()
    q6 = Queue()

    q1.put([BASE_ADDRESS + '/cardano_TXs_1.csv', True, False, False])
    q2.put([BASE_ADDRESS + '/cardano_TXs_2.csv', True, False, False])
    q3.put([BASE_ADDRESS + '/cardano_TXs_3.csv', True, False, False])
    q4.put([BASE_ADDRESS + '/cardano_TXs_4.csv', True, False, False])
    q5.put([BASE_ADDRESS + '/cardano_TXs_5.csv', True, False, False])
    q6.put([BASE_ADDRESS + '/cardano_TXs_6.csv', True, False, False])

    # Create Processes:
    p1 = mp.Process(target=generate_parents_array, args=(q1,))
    p2 = mp.Process(target=generate_parents_array, args=(q2,))
    p3 = mp.Process(target=generate_parents_array, args=(q3,))
    p4 = mp.Process(target=generate_parents_array, args=(q4,))
    p5 = mp.Process(target=generate_parents_array, args=(q5,))
    p6 = mp.Process(target=generate_parents_array, args=(q6,))

    # Start Processes:
    p1.start()
    p2.start()
    p3.start()
    p4.start()
    p5.start()
    p6.start()

    # Wait for Processes to finish:
    p1.join()
    p2.join()
    p3.join()
    p4.join()
    p5.join()
    p6.join()

    print('----------------------')
    output_filename_1 = q1.get()
    parents_1    = load_file_to_array   (output_filename_1[0])
    graghEdges_1 = load_file_to_array_2D(output_filename_1[1])
    print('parents_1 and graghEdges_1 loaded!')

    output_filename_2 = q2.get()
    parents_2    = load_file_to_array   (output_filename_2[0])
    graghEdges_2 = load_file_to_array_2D(output_filename_2[1])
    print('parents_2 and graghEdges_2 loaded!')

    output_filename_3 = q3.get()
    parents_3    = load_file_to_array   (output_filename_3[0])
    graghEdges_3 = load_file_to_array_2D(output_filename_3[1])
    print('parents_3 and graghEdges_3 loaded!')

    output_filename_4 = q4.get()
    parents_4    = load_file_to_array   (output_filename_4[0])
    graghEdges_4 = load_file_to_array_2D(output_filename_4[1])
    print('parents_4 and graghEdges_4 loaded!')

    output_filename_5 = q5.get()
    parents_5    = load_file_to_array   (output_filename_5[0])
    graghEdges_5 = load_file_to_array_2D(output_filename_5[1])
    print('parents_5 and graghEdges_5 loaded!')

    output_filename_6 = q6.get()
    parents_6    = load_file_to_array   (output_filename_6[0])
    graghEdges_6 = load_file_to_array_2D(output_filename_6[1])
    print('parents_6 and graghEdges_6 loaded!')


##########################################################################################
print('----------------------')
et = datetime.datetime.now() - ct
print("Total elapsed time (Fill Parents_Arrays with Heuristics for all CSV Files): ", et)


##########################################################################################
print('----------------------')
print('done!')



In [ ]:
# Create and Fill "Parents_Merged Arrays":


print('----------------------')

# ct stores current time
ct = datetime.datetime.now()
print("current time: ", ct)

##########################################################################################
# Initialize parents array:
parents_merged = np.array([[0]] * unique_payment_addresses_len)
for i in range(unique_payment_addresses_len):
    parents_merged[i] = i


##########################################################################################
# parents_merged = "parents_1 + parents_2 + parents_3 + parents_4 + parents_5 + parents_6":

'''
print('----------------------')

resolveAll (parents_1)
print('parents_1[] resolved!')

resolveAll (parents_2)
print('parents_2[] resolved!')

resolveAll (parents_3)
print('parents_3[] resolved!')

resolveAll (parents_4)
print('parents_4[] resolved!')

resolveAll (parents_5)
print('parents_5[] resolved!')

resolveAll (parents_6)
print('parents_6[] resolved!')
'''

'''
print('----------------------')

merge_parents(parents_1, parents_merged)
print('parents_1[] merged!')

merge_parents(parents_2, parents_merged)
print('parents_2[] merged!')

merge_parents(parents_3, parents_merged)
print('parents_3[] merged!')

merge_parents(parents_4, parents_merged)
print('parents_4[] merged!')

merge_parents(parents_5, parents_merged)
print('parents_5[] merged!')

merge_parents(parents_6, parents_merged)
print('parents_6[] merged!')
'''

##########################################################################################
# parents_merged = "parents_heur2_array":


'''
print('----------------------')

resolveAll (parents_heur2_array)
print('parents_heur2_array resolved!')
'''

'''
print('----------------------')

merge_parents(parents_heur2_array, parents_merged)
print('parents_heur2_array merged!')
'''


##########################################################################################
# parents_merged =   "parents_heur2_array":
#                  + "parents_1 + parents_2 + parents_3 + parents_4 + parents_5 + parents_6":

print('----------------------')

# Load parents_merged from file:
file_name = BASE_ADDRESS + '/parentsList_Heuristic1noSC__Cardano_TXs_All__2023-02-25_223712.txt'
parents_heur1_array = load_file_to_array (file_name)

file_name = BASE_ADDRESS + '/parentsList_Heuristic2__Cardano_TXs_All__2023-03-26_105842.txt'
parents_heur2_array = load_file_to_array (file_name)

'''
resolveAll (parents_heur1_array)
print('parents_heur1_array resolved!')

resolveAll (parents_heur2_array)
print('parents_heur2_array resolved!')
'''

merge_parents(parents_heur1_array, parents_merged)
print('parents_heur1_array merged!')

merge_parents(parents_heur2_array, parents_merged)
print('parents_heur2_array merged!')


##########################################################################################
print('----------------------')

resolveAll (parents_merged)
print('parents_merged[] resolved!')


##########################################################################################
print('----------------------')
et = datetime.datetime.now() - ct
print("Total elapsed time (Fill Parents_Merged Array): ", et)

##########################################################################################
print('----------------------')
print('done!')



In [ ]:
# Store parents_merged into file:

print('----------------------')

ct = datetime.datetime.now()
curr_timestamp = str(ct)[0:10] + '_' + str(ct)[11:13] + str(ct)[14:16] + str(ct)[17:19]

#output_filename = BASE_ADDRESS + '/parentsList_Heuristic1noSC__Cardano_TXs_All__' + curr_timestamp + '.txt'
#output_filename = BASE_ADDRESS + '/parentsList_Heuristic2__Cardano_TXs_All__' + curr_timestamp + '.txt'
output_filename = BASE_ADDRESS + '/parentsList_Heuristic1noSC_AND_Heuristic2__Cardano_TXs_All__' + curr_timestamp + '.txt'



print('output_filename = ', output_filename)
store_array_to_file (parents_merged, output_filename)

##########################################################################################
print('----------------------')
print('done!')



In [ ]:
# Create and Fill "Clustering Array":

print('----------------------')

# ct stores current time
ct = datetime.datetime.now()
print("current time: ", ct)


clustering_array = np.array([0] * unique_payment_addresses_len)

num_of_clusters = remapClusterIds (parents_merged, clustering_array)


##########################################################################################
print('----------------------')
print('Length of \"clustering_array\" = ', len(clustering_array))
print('Number of Clusters           = '  , len(np.unique(clustering_array)))
print('Number of Clusters           = '  , max(clustering_array)+1)
print('Number of Clusters           = '  , num_of_clusters)


print('----------------------')
print('clustering_array = ', clustering_array)

##########################################################################################
print('----------------------')
et = datetime.datetime.now() - ct
print("Total elapsed time (Fill Clustering Array with Heuristics): ", et)

##########################################################################################
print('----------------------')
print('done!')



In [6]:
# Store/Load clustering_array into file:

print('----------------------')

ct = datetime.datetime.now()
curr_timestamp = str(ct)[0:10] + '_' + str(ct)[11:13] + str(ct)[14:16] + str(ct)[17:19]


# Store clustering_array[] into file:
'''
#output_filename = BASE_ADDRESS + '/clusteringArrayList_Heuristic1noSC__Cardano_TXs_All__' + curr_timestamp + '.txt'
#output_filename = BASE_ADDRESS + '/clusteringArrayList_Heuristic2__Cardano_TXs_All__' + curr_timestamp + '.txt'
output_filename = BASE_ADDRESS + '/clusteringArrayList_Heuristic1noSC_AND_Heuristic2__Cardano_TXs_All__' + curr_timestamp + '.txt'

print('output_filename = ', output_filename)
store_array_to_file (clustering_array, output_filename)
'''


# Load clustering_array[] from file:

#file_name = BASE_ADDRESS + '/clusteringArrayList_Heuristic1noSC__Cardano_TXs_All__2023-02-25_223957.txt'
#file_name = BASE_ADDRESS + '/clusteringArrayList_Heuristic2__Cardano_TXs_All__2023-03-26_110150.txt'
file_name = BASE_ADDRESS + '/clusteringArrayList_Heuristic1noSC_AND_Heuristic2__Cardano_TXs_All__2023-03-26_141212.txt'

clustering_array = load_file_to_array (file_name)


##########################################################################################
print('----------------------')
print('done!')


----------------------
start time (Load /local/scratch/exported/blockchain_parsed/cardano_mostafa/clusteringArrayList_Heuristic1noSC_AND_Heuristic2__Cardano_TXs_All__2023-03-26_141212.txt to Array):  2023-04-07 11:23:09.755594
elapsed time (Load /local/scratch/exported/blockchain_parsed/cardano_mostafa/clusteringArrayList_Heuristic1noSC_AND_Heuristic2__Cardano_TXs_All__2023-03-26_141212.txt to Array):  0:00:02.532619
----------------------
done!


In [ ]:
# Merge graghEdges_ Arrays:

def merge_graphEdges(graghEdges_array, graghEdges_merged):
    if (len(graghEdges_array) != len(graghEdges_merged)):
        print('merge_graphEdges Error: -1 (Length)')
        return -1
    
    for i in tqdm(range(len(graghEdges_merged))):
        graghEdges_merged[i].extend(graghEdges_array[i])
    
    return





print('----------------------')
# ct stores current time
ct = datetime.datetime.now()
print("current time: ", ct)

##########################################################################################

graphEdges_merged = [[] for _ in range(unique_payment_addresses_len)]


merge_graphEdges(graghEdges_1, graphEdges_merged)
print('graghEdges_1[] merged!')


merge_graphEdges(graghEdges_2, graphEdges_merged)
print('graghEdges_2[] merged!')

merge_graphEdges(graghEdges_3, graphEdges_merged)
print('graghEdges_3[] merged!')

merge_graphEdges(graghEdges_4, graphEdges_merged)
print('graghEdges_4[] merged!')

merge_graphEdges(graghEdges_5, graphEdges_merged)
print('graghEdges_5[] merged!')

merge_graphEdges(graghEdges_6, graphEdges_merged)
print('graghEdges_6[] merged!')


##########################################################################################
print('----------------------')
et = datetime.datetime.now() - ct
print("Total elapsed time (Fill Clustering Array with Heuristics): ", et)

##########################################################################################
print('----------------------')
print('done!')




In [ ]:
# Store graphEdges_merged into file:

print('----------------------')

ct = datetime.datetime.now()
curr_timestamp = str(ct)[0:10] + '_' + str(ct)[11:13] + str(ct)[14:16] + str(ct)[17:19]
#output_filename = BASE_ADDRESS + '/graphEdgesArrayList_Heuristic1_LinkToFirstAddressInTX__Cardano_TXs_All__' + curr_timestamp + '.txt'
output_filename = BASE_ADDRESS + '/graphEdgesArrayList_Heuristic1noSC_LinkToALLAddressesInTX__Cardano_TXs_All__' + curr_timestamp + '.txt'
print('output_filename = ', output_filename)

store_array_to_file_2D (graphEdges_merged, output_filename)

##########################################################################################
print('----------------------')
print('done!')